In [1]:
import pandas as pd
import numpy as np
from pyensembl import EnsemblRelease

In [4]:
gene_encodings = pd.read_csv("Merged_Gene_freq_encoding.csv")

/home/lava-gpu/anaconda3/envs/ml-lava/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
gene_encodings

,Unnamed: 0,Unnamed: 0.1,DonorIDs,CancerType,ENSG00000000003,ENSG00000000005,ENSG00000000419,ENSG00000000457,ENSG00000000460,ENSG00000000938,...,ENSG00000273456,ENSG00000273466,ENSG00000273471,ENSG00000273476,ENSG00000273477,ENSG00000273478,ENSG00000273483,ENSG00000273485,ENSG00000273488,ENSG00000273489
0,0,0,DO48566BLCA-US,BLCA-US,0.0,0.0,0.0,2.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,DO223588BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2,2,DO51948BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,3,DO514BLCA-US,BLCA-US,0.0,0.0,0.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,4,DO474BLCA-US,BLCA-US,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5863,2499,5863,NaN,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5864,2500,5864,NaN,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5865,2501,5865,NaN,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5866,2502,5866,NaN,Healthy,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
def read_pathways_csv(filename):
    pathway_mappings = {}
    with open(filename, "r") as csv_file:
        for line in csv_file:
            stripped = line.replace('"', '')
            tokens = stripped.split(',')
            gene_names = [name.strip() for name in tokens[6:]]
            pathway_mappings[tokens[0]] = gene_names
    return pathway_mappings
        

In [20]:
pathway_mappings = read_pathways_csv("pathways.csv")

In [22]:
len(pathway_mappings.items())

1657

In [23]:
def build_reverse_pathway_lookup(pathway_mappings):
    reverse_lookup_table = {}
    
    for pathway, gene_names in pathway_mappings.items():
        for name in gene_names:
            try:
                reverse_lookup_table[name].append(pathway)
            except Exception:
                reverse_lookup_table[name] = [pathway]
    return reverse_lookup_table


In [24]:
reverse_pathway_lookup = build_reverse_pathway_lookup(pathway_mappings)

In [25]:
len(reverse_pathway_lookup.items())

8522

In [27]:
ensembl_data = EnsemblRelease(77)

In [28]:
gene = ensembl_data.gene_by_id('ENSG00000000003')

In [29]:
gene

Gene(gene_id='ENSG00000000003', gene_name='TSPAN6', biotype='protein_coding', contig='X', start=100627109, end=100639991, strand='-', genome='GRCh38')

In [50]:
reverse_pathway_lookup['CBX2']

['SUMOylation of transcription cofactors',
 'SUMOylation of RNA binding proteins',
 'SUMOylation of DNA methylation proteins',
 'SUMOylation of DNA damage response and repair proteins',
 'SUMOylation of chromatin organization proteins',
 'SUMOylation',
 'SUMO E3 ligases SUMOylate target proteins',
 'Post-translational protein modification',
 'Oxidative Stress Induced Senescence',
 'Cellular Senescence',
 'Cellular responses to stress']

In [33]:
from bioservices.kegg import KEGG

Creating directory /home/lava-gpu/.config/bioservices 


In [35]:
gene_ids = [col_name for col_name in gene_encodings.columns if col_name.startswith("ENSG")]

In [37]:
k = KEGG(verbose=False)

Creating directory /home/lava-gpu/.cache/bioservices 
Welcome to Bioservices
It looks like you do not have a configuration file.
We are creating one with default values in /home/lava-gpu/.config/bioservices/bioservices.cfg .
Done


In [38]:
k.find("pathway", gene.gene_name)

'\n'

In [41]:
k.find("hsa", "TSPAN6")

'hsa:7105\tTSPAN6, T245, TM4SF6, TSPAN-6; tetraspanin 6\n'

In [47]:
res = k.get_pathway_by_gene("7105", "hsa")

In [51]:
zs = np.zeros(10)
zs[0] += 1
zs

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [58]:
def build_gene_name_mappings(gene_ids):
    gene_name_mappings = {}
    for gene_id in gene_ids:
        try:
            gene = ensembl_data.gene_by_id(gene_id)
            name = gene.gene_name
            gene_name_mappings[gene_id] = name
        except Exception:
            None
    return gene_name_mappings

In [59]:
gene_name_table = build_gene_name_mappings(gene_ids)

In [64]:
def build_pathway_encodings(gene_encodings, pathway_lookup, reverse_lookup, gene_name_table):
    gene_ids = gene_ids = [col_name for col_name in gene_encodings.columns if col_name.startswith("ENSG")]
    gene_data = gene_encodings[gene_ids].to_numpy()
    pathway_encodings_table = {}
    for key in pathway_lookup.keys():
        pathway_encodings_table[key] = np.zeros(gene_data.shape[0])
    
    for i in range(gene_data.shape[0]):
        for j in range(gene_data.shape[1]):
            if gene_data[i][j] > 0:
                try:
                    gene_id = gene_ids[i]
                    name = gene_name_table[gene_id]
                    pathways = reverse_lookup[name]
                    for pathway in pathways:
                        pathway_encodings_table[pathway][i] += gene_data[i][j]
                except Exception:
                    None
    pathway_encodings = pd.DataFrame.from_dict(pathway_encodings_table)
    pathway_encodings["CancerType"] = gene_encodings["CancerType"]
    return pathway_encodings

In [65]:
pathway_encodings = build_pathway_encodings(
    gene_encodings, pathway_mappings, reverse_pathway_lookup, gene_name_table
)

In [66]:
pathway_encodings

,Zinc transporters,Zinc influx into cells by the SLC39 gene family,Zinc efflux and compartmentalization by the SLC30 family,ZBP1(DAI) mediated induction of type I IFNs,YAP1- and WWTR1 (TAZ)-stimulated gene expression,Xenobiotics,XBP1(S) activates chaperone genes,WNT mediated activation of DVL,WNT ligand biogenesis and trafficking,WNT5A-dependent internalization of FZD4,...,Acetylcholine regulates insulin secretion,Acetylcholine Neurotransmitter Release Cycle,Acetylcholine binding and downstream events,ABO blood group biosynthesis,ABC transporters in lipid homeostasis,ABC-family proteins mediated transport,Abacavir transport and metabolism,Abacavir transmembrane transport,Abacavir metabolism,CancerType
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLCA-US
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLCA-US
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLCA-US
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLCA-US
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BLCA-US
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Healthy
5864,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Healthy
5865,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Healthy
5866,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Healthy


In [67]:
for column in pathway_encodings.columns:
    if column != "CancerType":
        print("{}: {}".format(column, sum(pathway_encodings[column])))

Zinc transporters: 1625.0
Zinc influx into cells by the SLC39 gene family: 0.0
Zinc efflux and compartmentalization by the SLC30 family: 1625.0
ZBP1(DAI) mediated induction of type I IFNs: 6667.0
YAP1- and WWTR1 (TAZ)-stimulated gene expression: 7065.0
Xenobiotics: 6407.0
XBP1(S) activates chaperone genes: 19589.0
WNT mediated activation of DVL: 8114.0
WNT ligand biogenesis and trafficking: 20332.0
WNT5A-dependent internalization of FZD4: 2425.0
WNT5A-dependent internalization of FZD2: 2022.0
Wax biosynthesis: 0.0
Wax and plasmalogen biosynthesis: 1627.0
VxPx cargo-targeting to cilium: 6450.0
Voltage gated Potassium channels: 36026.0
VLDLR internalisation and degradation: 2232.0
VLDL clearance: 1426.0
VLDL assembly: 117.0
Vitamins B6 activation to pyridoxal phosphate: 0.0
Vitamins: 11994.0
Vitamin D (calciferol) metabolism: 5103.0
Vitamin C (ascorbate) metabolism: 2250.0
Vitamin B5 (pantothenate) metabolism: 5295.0
Vitamin B2 (riboflavin) metabolism: 1087.0
Vitamin B1 (thiamin) metabol

The canonical retinoid cycle in rods (twilight vision): 8438.0
The AIM2 inflammasome: 44.0
The activation of arylsulfatases: 3778.0
TGF-beta receptor signaling in EMT (epithelial to mesenchymal transition): 18540.0
TGF-beta receptor signaling activates SMADs: 28524.0
TFAP2A acts as a transcriptional repressor during retinoic acid induced cell differentiation: 1486.0
TFAP2 (AP-2) family regulates transcription of other transcription factors: 62.0
TFAP2 (AP-2) family regulates transcription of growth factors and their receptors: 1997.0
TFAP2 (AP-2) family regulates transcription of cell cycle factors: 3201.0
Tetrahydrobiopterin (BH4) synthesis: 2056.0
TET1: 0.0
Termination of translesion DNA synthesis: 8970.0
Termination of O-glycan biosynthesis: 5302.0
Terminal pathway of complement: 6469.0
Telomere Maintenance: 11130.0
Telomere Extension By Telomerase: 0.0
Telomere C-strand synthesis initiation: 3971.0
Telomere C-strand (Lagging Strand) Synthesis: 10344.0
TCR signaling: 36539.0
TCF dep

Scavenging of heme from plasma: 6077.0
Scavenging by Class H Receptors: 1750.0
Scavenging by Class F Receptors: 1883.0
Scavenging by Class B Receptors: 1771.0
Scavenging by Class A Receptors: 5009.0
RUNX3 regulates YAP1-mediated transcription: 547.0
RUNX3 regulates WNT signaling: 1747.0
RUNX3 regulates p14-ARF: 17897.0
RUNX3 regulates NOTCH signaling: 7495.0
RUNX3 Regulates Immune Response and Cell Migration: 2275.0
RUNX3 regulates CDKN1A transcription: 17058.0
RUNX3 regulates BCL2L11 (BIM) transcription: 1691.0
RUNX2 regulates osteoblast differentiation: 8272.0
RUNX2 regulates genes involved in differentiation of myeloid cells: 1844.0
RUNX2 regulates genes involved in cell migration: 1844.0
RUNX2 regulates chondrocyte maturation: 2204.0
RUNX2 regulates bone development: 8632.0
RUNX1 regulates transcription of genes involved in WNT signaling: 705.0
RUNX1 regulates transcription of genes involved in interleukin signaling: 3320.0
RUNX1 regulates transcription of genes involved in differe

Regulation of expression of SLITs and ROBOs: 96945.0
Regulation of cytoskeletal remodeling and cell spreading by IPP complex components: 1655.0
Regulation of cortical dendrite branching: 155.0
Regulation of Complement cascade: 22374.0
Regulation of commissural axon pathfinding by SLIT and ROBO: 1655.0
Regulation of cholesterol biosynthesis by SREBP (SREBF): 24933.0
Regulation of beta-cell development: 12447.0
Regulation of Apoptosis: 21885.0
Regulation of APC/C activators between G1/S and early anaphase: 48272.0
Regulation of activated PAK-2p34 by proteasome mediated degradation: 21732.0
Regulation of actin dynamics for phagocytic cup formation: 12106.0
Regulation by c-FLIP: 10149.0
Regulated proteolysis of p75NTR: 29412.0
Regulated Necrosis: 11849.0
Reelin signalling pathway: 153.0
Reduction of cytosolic Ca++ levels: 2397.0
Recycling pathway of L1: 5800.0
Recycling of eIF2:GDP: 6113.0
Recycling of bile acids and salts: 5394.0
Recruitment of NuMA to mitotic centrosomes: 26672.0
Recruit

Ovarian tumor domain proteases: 8089.0
Other semaphorin interactions: 4606.0
Other interleukin signaling: 10301.0
Organic cation transport: 1905.0
Organic cation/anion/zwitterion transport: 1905.0
Organic anion transporters: 3052.0
Organic anion transport: 0.0
Orexin and neuropeptides FF and QRFP bind to their respective receptors: 1682.0
Orc1 removal from chromatin: 31668.0
Opsins: 3442.0
Opioid Signalling: 43089.0
Oncogene Induced Senescence: 16030.0
O-linked glycosylation of mucins: 11947.0
O-linked glycosylation: 13635.0
Oligomerization of connexins into connexons: 0.0
Olfactory Signaling Pathway: 3671.0
O-glycosylation of TSR domain-containing proteins: 1438.0
OAS antiviral response: 4841.0
O2/CO2 exchange in erythrocytes: 2405.0
Nucleotide salvage: 1791.0
Nucleotide-like (purinergic) receptors: 158.0
Nucleotide Excision Repair: 37896.0
Nucleotide-binding domain: 12013.0
Nucleosome assembly: 10810.0
Nucleobase catabolism: 5485.0
Nucleobase biosynthesis: 2618.0
Nuclear signaling by

Metabolism of lipids: 282070.0
Metabolism of ingested SeMet: 5655.0
Metabolism of ingested H2SeO4 and H2SeO3 into H2Se: 0.0
Metabolism of folate and pterines: 2134.0
Metabolism of fat-soluble vitamins: 19651.0
Metabolism of cofactors: 5196.0
Metabolism of carbohydrates: 97571.0
Metabolism of Angiotensinogen to Angiotensins: 11482.0
Metabolism of amino acids and derivatives: 135709.0
Metabolism of amine-derived hormones: 5183.0
Membrane Trafficking: 294047.0
Melanin biosynthesis: 1943.0
Meiotic synapsis: 9948.0
Meiotic recombination: 11817.0
Meiosis: 21626.0
MECP2 regulates transcription of neuronal ligands: 3415.0
MECP2 regulates transcription of genes involved in GABA signaling: 0.0
MECP2 regulates transcription factors: 1873.0
MECP2 regulates neuronal receptors and channels: 7435.0
MASTL Facilitates Mitotic Progression: 1514.0
MAPK targets/ Nuclear events mediated by MAP kinases: 18326.0
MAP kinase activation: 27843.0
MAPK family signaling cascades: 117829.0
MAPK6/MAPK4 signaling: 32

GRB2:SOS provides linkage to MAPK signaling for Integrins: 7396.0
GPVI-mediated activation cascade: 4976.0
G-protein mediated events: 27199.0
G protein gated Potassium channels: 5090.0
G-protein beta:gamma signalling: 1664.0
G-protein activation: 12879.0
GPCR ligand binding: 158935.0
GPCR downstream signalling: 255252.0
GP1b-IX-V activation signalling: 1467.0
Golgi-to-ER retrograde transport: 42311.0
Golgi Cisternae Pericentriolar Stack Reorganization: 4965.0
Golgi Associated Vesicle Biogenesis: 13912.0
Glyoxylate metabolism and glycine degradation: 15451.0
Glycosphingolipid metabolism: 8862.0
Glycosaminoglycan metabolism: 45779.0
Glycoprotein hormones: 3644.0
Glycolysis: 31217.0
Glycogen synthesis: 6739.0
Glycogen metabolism: 8438.0
Glycogen breakdown (glycogenolysis): 1761.0
Glycine degradation: 313.0
Glycerophospholipid biosynthesis: 38570.0
Glutathione synthesis and recycling: 2377.0
Glutathione conjugation: 3180.0
Glutamate Neurotransmitter Release Cycle: 29695.0
Glutamate binding

Drug-mediated inhibition of ERBB2 signaling: 3929.0
Downstream TCR signaling: 29148.0
Downstream signal transduction: 13497.0
Downstream signaling of activated FGFR4: 14372.0
Downstream signaling of activated FGFR3: 13005.0
Downstream signaling of activated FGFR2: 15079.0
Downstream signaling of activated FGFR1: 18046.0
Downstream signaling events of B Cell Receptor (BCR): 26830.0
Downregulation of TGF-beta receptor signaling: 26820.0
Downregulation of SMAD2/3:SMAD4 transcriptional activity: 8204.0
Downregulation of ERBB4 signaling: 2566.0
Downregulation of ERBB2 signaling: 8754.0
Downregulation of ERBB2:ERBB3 signaling: 1574.0
Dopamine receptors: 934.0
Dopamine Neurotransmitter Release Cycle: 5387.0
Dopamine clearance from the synaptic cleft: 208.0
DNA strand elongation: 12310.0
DNA Replication Pre-Initiation: 44623.0
DNA replication initiation: 3971.0
DNA methylation: 1375.0
DNA Double Strand Break Response: 15245.0
DNA Double-Strand Break Repair: 38269.0
DNA Damage Reversal: 1927.0


Calcineurin activates NFAT: 1708.0
Ca-dependent events: 16053.0
Ca2+ pathway: 8477.0
Ca2+ activated K+ channels: 1701.0
B-WICH complex positively regulates rRNA expression: 12646.0
Butyrophilin (BTN) family interactions: 7859.0
Butyrate Response Factor 1 (BRF1) binds and destabilizes mRNA: 8307.0
Breakdown of the nuclear lamina: 1583.0
Branched-chain amino acid catabolism: 3527.0
BMAL1:CLOCK: 16846.0
Blood group systems biosynthesis: 8998.0
Biotin transport and metabolism: 3727.0
Biosynthesis of the N-glycan precursor (dolichol lipid-linked oligosaccharide: 27965.0
Biosynthesis of specialized proresolving mediators (SPMs): 5281.0
Biosynthesis of protectins: 1526.0
Biosynthesis of maresins: 2104.0
Biosynthesis of maresin-like SPMs: 98.0
Biosynthesis of E-series 18(S)-resolvins: 1671.0
Biosynthesis of E-series 18(R)-resolvins: 1671.0
Biosynthesis of EPA-derived SPMs: 1778.0
Biosynthesis of D-series resolvins: 1671.0
Biosynthesis of DPAn-3 SPMs: 1796.0
Biosynthesis of DPA-derived SPMs: 17

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [68]:
pathway_encodings.to_csv("pathway_encodings_full.csv")